<table>
 <tr align=left><td><img align=left src="https://mirrors.creativecommons.org/presskit/buttons/80x15/png/by.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Kyle T. Mandli. Adapted for CS/MATH 3414 by Arash Sarshar. Shared under the same licenses.</td>
 <td> </td>
</table>

In [8]:
from __future__ import print_function

%matplotlib inline
import numpy
import scipy.linalg
import matplotlib.pyplot as plt

# Solving linear systems (direct methods)


Direct solvers use ``matrix factorizations``. A process of breaking a matrix into a structured matrix product. In this section, we will learn about `LU factorization`. 


$$ 
\begin{align*}
A &= LU,\\
Ax &= LU x = b,
\end{align*}
$$

$$
    \begin{bmatrix}
    1\\ 5 \\ 9
    \end{bmatrix} = 
    \begin{bmatrix}
    3 & 7 & 11 \\
    3 & 8 & 14 \\
    1 & 2 & 3
    \end{bmatrix} 
    \begin{bmatrix}
    x_1\\ x_2 \\ x_3
    \end{bmatrix}    
    = \begin{bmatrix}
    1 & 0 & 0 \\
    1 & 1 & 0 \\
     \frac{1}{3}& -\frac{1}{3} & 1
    \end{bmatrix}
    \underbrace{
    \begin{bmatrix}
    3 & 7 & 11 \\
    0 & 1 & 3 \\
    0 & 0 & \frac{1}{3}
    \end{bmatrix}
    \begin{bmatrix}
    x_1\\ x_2 \\ x_3
    \end{bmatrix} 
    }_{z = U x} \,,
$$


$$
\begin{align*}
1 &= z_1, \\ 
5 &= z_1 + z_2, \\ 
9 &= \frac{1}{3} z_1 -  \frac{1}{3} z_2 + z_3.\\[20pt]
z_1 &= 3 x_1 + 7 x_2 + 11 x_3, \\ 
z_2 &= x_2 + 3 x_3, \\ 
z_3 &= \frac{1}{3} x_3. 
\end{align*}
$$


# Gaussian Elimination

One of the is the processes that can generate $LU$ factorizations is ``Gaussian elimination``. This process transforms a  linear system through a series of operations into one that is triangular and hence staightforward to solve.  These series of operations can be written as a sequence of successive matrix-matrix multiplications by lower triangular matrices.  Letting these matrices be $L_j$ and the resulting upper triangular matrix be $U$ we can write this as

$$
    \overbrace{L_{m-1}\, \cdots L_2\, L_1}^{L^{-1}} A = U.
$$

Labeling the effective operation as $L^{-1}$, we can move it to the other side of the equation and we see that in fact what we have done is computed the  $LU$ factorization of the matrix $A$. If we consider the linear system $Ax = b$, as we transform the matrix $A$ inro  upper triangulaur shape, we can apply the same transformations to the right hand side vectore to keep the system  the  same:

$$
\begin{align*}
    A x &= b, \\
    L_{m-1}\, \cdots L_2\, L_1\, Ax &=   L_{m-1}\, \cdots L_2\, L_1\, b = Ux.
\end{align*}
$$

### Example

As an example of this process lets consider the matrix
$$
    A = \begin{bmatrix}
        2 & 1 & 1 & 0 \\
        4 & 3 & 3 & 1 \\
        8 & 7 & 9 & 5 \\
        6 & 7 & 9 & 8
    \end{bmatrix}.
$$

Lets create a matrix operation that introduces one zero below the first row of A:

$$
    \hat L = \begin{bmatrix}
         1 &   &   &  \\
        -2 & 1 &   &  \\
         &   & 1 &  \\
         &   &   & 1
    \end{bmatrix} , \quad
        A = \begin{bmatrix}
        2 & 1 & 1 & 0 \\
        \color{red}{4} & 3 & 3 & 1 \\
        \color{red}{8} & 7 & 9 & 5 \\
        \color{red}{6} & 7 & 9 & 8
    \end{bmatrix} \quad  
    \hat L A = \begin{bmatrix}
        2 & 1 & 1 & 0 \\
        0  & 1 & 1 & 1 \\
        \color{red}{8}  & 7 & 9 & 5 \\
        \color{red}{6}  & 7 & 9 & 8
    \end{bmatrix}.
$$

The first step is to remove the values in the first column below the diagonal, such that

$$
     A = \begin{bmatrix}
        2 & 1 & 1 & 0 \\
        \color{red}{4} & 3 & 3 & 1 \\
        \color{red}{8} & 7 & 9 & 5 \\
        \color{red}{6} & 7 & 9 & 8
    \end{bmatrix} 
     {\rightarrow} \quad  L_1 A = \begin{bmatrix}
        2 & 1 & 1 & 0 \\
        0  & 1 & 1 & 1 \\
        0  & 3 & 5 & 5 \\
        0  & 4 & 6 & 8
    \end{bmatrix}.
$$

So to make zero out all elements in the first column below the diagonal,  we can multiply by the matrix

$$
    L_1 = \begin{bmatrix}
         1 &   &   &  \\
        -2 & 1 &   &  \\
        -4 &   & 1 &  \\
        -3 &   &   & 1
    \end{bmatrix} \text{ so that } L_1 A = \begin{bmatrix}
        2 & 1 & 1 & 0 \\
        0  & {1} & 1 & 1 \\
        0  & \color{red}{3} & 5 & 5 \\
        0  & \color{red}{4} & 6 & 8
    \end{bmatrix}.
$$

The next step is to remove the values below the diagonal of the second column.  This can be done with

$$
    L_2 = \begin{bmatrix}
         1 &    &   &   \\
           &  1 &   &   \\
           & -3 & 1 &   \\
           & -4 &   & 1
    \end{bmatrix} \text{ so that } L_2 L_1 A = \begin{bmatrix}
        2 & 1 & 1 & 0 \\
          & 1 & 1 & 1 \\
          &   & 2 & 2 \\
          &   & \color{red}{2} & 4
    \end{bmatrix}.
$$

Finally we multiply $A$ by $L_3$ defined as


$$
    L_3 = \begin{bmatrix}
         1 &   &    &   \\
           & 1 &    &   \\
           &   &  1 &   \\
           &   & -1 & 1
    \end{bmatrix} \text{ so that } L_3 L_2 L_1 A = \begin{bmatrix}
        2 & 1 & 1 & 0 \\
          & 1 & 1 & 1 \\
          &   & 2 & 2 \\
          &   &   & 2
    \end{bmatrix}
$$


completing the factorization with
$$
\begin{align*}
    L^{-1} = L_3 L_2 L_1 &= \begin{bmatrix}
         1 &   &    &   \\
           & 1 &    &   \\
           &   &  1 &   \\
           &   & -1 & 1
    \end{bmatrix}
    \begin{bmatrix}
         1 &    &   &   \\
           &  1 &   &   \\
           & -3 & 1 &   \\
           & -4 &   & 1
    \end{bmatrix} 
    \begin{bmatrix}
         1 &   &   &   \\
        -2 & 1 &   &   \\
        -4 &   & 1 &   \\
        -3 &   &   & 1
    \end{bmatrix}, \\
    U &= \begin{bmatrix}
        2 & 1 & 1 & 0 \\
          & 1 & 1 & 1 \\
          &   & 2 & 2 \\
          &   &   & 2
    \end{bmatrix}
    \end{align*}
$$

Inverting $L^{-1}$ we can finally write $A$ as
$$
    A =
    \begin{bmatrix}
         1 &   &   &   \\
         2 & 1 &   &   \\
         4 & 3 & 1 &   \\
         3 & 4 & 1 & 1
    \end{bmatrix}\begin{bmatrix}
        2 & 1 & 1 & 0 \\
          & 1 & 1 & 1 \\
          &   & 2 & 2 \\
          &   &   & 2
    \end{bmatrix}
$$

Inverting $L^{-1}$ ?

$$
L^{-1} \begin{bmatrix} l_1 & l_2 &\cdots& l_m  \end{bmatrix} = I = \begin{bmatrix} e_1 & e_2 &\cdots& e_m  \end{bmatrix}
$$

## Pivoting

Pivoting allows us  to perform $LU$ factorization in a more robust way. As a simple example take the matrix

$$
    A = \begin{bmatrix} 0 & 1 \\ 1 & 1 \end{bmatrix},  \quad  \hat A = \begin{bmatrix} 1 & 1 \\ 0 & 1 \end{bmatrix}.
$$

Without switch the rows Gaussian elimination would fail at the first step!  This is made more hazardous if we consider the matrix

$$
    A = \begin{bmatrix} 10^{-17} & 1 \\ 1 & 1 \end{bmatrix}.
$$

on a finite precision machine.  In principle any row and column can be **pivoted** so that at each step we have the maximum value being used (on the diagonal) to perform the operations that compose the matrices $L_j$.  In practice however we restrict ourselves to only pivoting rows of the matrix (called **partial pivoting**).

__Reminder__: $ \text{float}(x/y) = x/y (1 + \varepsilon).$

Consider again the example from above and switch the 1st and 3rd rows using the criteria that we always want to use the largest value to do perform the reduction.  Defining the first pivot matrix as

$$
    P_1 = \begin{bmatrix}
          &   & 1 &   \\
          & 1 &   &   \\
        1 &   &   &   \\
          &   &   & 1
    \end{bmatrix}, \quad 
     A = \begin{bmatrix}
        \color{blue}{2} & \color{blue}{1} & \color{blue}{1} & \color{blue}{0} \\
        {4} & 3 & 3 & 1 \\
        \boxed{\color{blue}{8}} & \color{blue}{7} & \color{blue}{9} & \color{blue}{5} \\
        {6} & 7 & 9 & 8
    \end{bmatrix},
    \quad
    P_1 A = \begin{bmatrix}
        8 & 7 & 9 & 5 \\
        4 & 3 & 3 & 1 \\
        2 & 1 & 1 & 0 \\
        6 & 7 & 9 & 8
    \end{bmatrix}
$$

Now defining the first $L_1$ as

$$
    L_1 = \begin{bmatrix}
        1 &   &   &   \\
        -\frac{1}{2} & 1 &   &   \\
        -\frac{1}{4} &   & 1 &   \\
        -\frac{3}{4} &   &   & 1
    \end{bmatrix} \quad \text{so that} \quad
    L_1 P_1 A = \begin{bmatrix}
        8 & 7 & 9 & 5 \\
          &\color{red}{ -\frac{1}{2}} & -\frac{3}{2} & -\frac{3}{2} \\
          & \color{red}{-\frac{3}{4}} & -\frac{5}{4} & -\frac{5}{4} \\
          & \boxed{\color{red}{\frac{7}{4}}} & \frac{9}{4} & \frac{17}{4}
    \end{bmatrix}.
$$

Again examining the remaining values in column 2 the maximum value lies in row 4 so we want to interchange this with the second row (note that we do not want to move the first row as that will bring non-zero values into the first column below the diagonal).

$$
    P_2 = \begin{bmatrix}
        1 &   &   &   \\
          &   &   & 1 \\
          &   & 1 &   \\
          & 1 &   &
    \end{bmatrix}  \quad \text{and} \quad 
    L_2 = \begin{bmatrix}
        1 &   &   &   \\
          & 1 &   &   \\
          & \frac{3}{7} & 1 &   \\
          & \frac{2}{7}  &   & 1
    \end{bmatrix}  \quad  \text{so that}  \quad 
    L_2 P_2 L_1 P_1 A = \begin{bmatrix}
        8 &            7 &            9 &             5 \\
          &  \frac{7}{4} &  \frac{9}{4} &  \frac{17}{4} \\
          &              & \color{red}{-\frac{2}{7}} &   \frac{4}{7} \\
          &              & \boxed{\color{red}{-\frac{6}{7}}} &  -\frac{2}{7}
    \end{bmatrix}.
$$

Finally
$$
    P_3 = \begin{bmatrix}
        1 &   &   &   \\
          & 1 &   &   \\
          &   &   & 1 \\
          &   & 1 &
    \end{bmatrix}  \quad \text{and} \quad 
    L_3 = \begin{bmatrix}
        1 &   &   &   \\
          & 1 &   &   \\
          &   & 1 &   \\
          &   & -\frac{1}{3} & 1
    \end{bmatrix} \quad \text{so that} \quad
    L_3 P_3 L_2 P_2 L_1 P_1 A = \begin{bmatrix}
        8 &            7 &            9 &             5 \\
          &  \frac{7}{4} &  \frac{9}{4} &  \frac{17}{4} \\
          &              & -\frac{6}{7} &  -\frac{2}{7} \\
          &              &              &   \frac{2}{3} \\
    \end{bmatrix}.
$$

Some observations about row permutation matrices:

$$
    P_3 = 
    \begin{bmatrix}
        1 &   &   &   \\
          & 1 &   &   \\
          &   &   & 1 \\
          &   & 1 &
    \end{bmatrix}, \quad  P_3^T  = 
    \begin{bmatrix}
        1 &   &   &   \\
          & 1 &   &   \\
          &   &   & 1 \\
          &   & 1 &
    \end{bmatrix}  
$$

* $P_j^T = P_j$
* $P_j^2 =  P_j P_j   = I$

### LU Factorization with Partial Pivoting

Pivot matrices are examples of `Permutation` matrices. They are permutations of rows (or colums) of the identity matrix. General permuation matriceshave the property that $P_j^{-1}=P_j^T$.

Can we find a single effective permutation for  the ``LU factorization`` such that
$
   L_2 P_2 L_1 P_1 A = U
$
can be written as  
$
(L'_2 L'_1)  P_2 P_1 A {=} U?
$


$$
    L_2 P_2 L_1 P_1 A = U,
$$

$$
    L_2 P_2 L_1 \underbrace{P_2^{-1} P_2}_I P_1 A = U,
$$


$$
    L_2 \underbrace{ P_2 L_1 P_2^{-1}}_{L_1'} P_2 P_1 A = U.
$$

We can extend this to multiple levels:

$$
    L'_3 = L_3, \quad L'_2 = P_3 L_2 P_3^{-1}, \quad \text{and} \quad L'_1 = P_3 P_2 L_1 P_2^{-1} P_3^{-1}.
$$

The new matrices $L'_j$ are permutations (in columns and rows) of $L_j$

In general then the $LU$ factorization with partial pivoting of the above example can be written as

$$
\begin{align*}
(L'_3 L'_2 L'_1)  P_3 P_2 P_1 A &= U,\\
P A &= LU,
\end{align*}
$$

$$
    \underbrace{\begin{bmatrix}
          &   & 1 &   \\
          &   &   & 1 \\
          & 1 &   &   \\
        1 &   &   & 
    \end{bmatrix}}_{P = P_3 P_2 P_1} \underbrace{\begin{bmatrix}
        2 & 1 & 1 & 0 \\
        4 & 3 & 3 & 1 \\
        8 & 7 & 9 & 5 \\
        6 & 7 & 9 & 8
    \end{bmatrix}}_{A} = 
    \underbrace{\begin{bmatrix}
        1           &              &              &   \\
        \frac{3}{4} &            1 &              &   \\
        \frac{1}{2} & -\frac{2}{7} &            1 &   \\
        \frac{1}{4} & -\frac{3}{7} & \frac{1}{3}  & 1 \\
    \end{bmatrix}}_{L = {(L'_3 L'_2 L'_1)^{-1}}}
    \underbrace{\begin{bmatrix}
        8 &            7 &            9 &             5 \\
          &  \frac{7}{4} &  \frac{9}{4} &  \frac{17}{4} \\
          &              & -\frac{6}{7} &  -\frac{2}{7} \\
          &              &              &   \frac{2}{3} \\
    \end{bmatrix}}_{U}
$$

The algorithm for the general factorization with partial pivoting then can be written as
```
U = A
L = I
P = I
for k = 1 to m - 1
    Select i >= k to maximize |u_{i,k}|
    P[k, :] <==> P[i, :]
    U[k, k:m] <==> U[i, k:m]
    L[k, 1:k-1] <==> L[i, 1:k-1]
    
    for j = k + 1 to m
        L[j, k] = U[j, k] / U[k, k]
        U[j, k:m] = U[j, k:m] - L[j, k] * U[k, k:m]
```
where `<==>` represents swapping of the two rows indicated.

## Solving $Ax = b$

To complete our discussion lets consider the solution of the linear system $Ax = b$.  We now have a factorization of the matrix $PA = LU$ so that the new system is $ LU x = Pb$ (note that pivoting is allowed as long as we also interchange the elements of $b$ via $P \cdot b$).  We can do this in two steps:

1. Compute the solution to $L y = P b$ using forward substitution.

1. Compute the solution to $U x = y$ using backward substitution.


### Forward Substitution

For forward substitution we proceed from the first row and progress downwards through the matrix.  We can then consider the general $i$th row with
$$
    L_{i,1} y_1 + L_{i,2} y_2 + \cdots + L_{i,i-1} y_{i-1} + y_i = b_i
$$
noting that we are using the fact that the matrix $L$ has 1 on its diagonal.  We can now solve for $y_i$ as
$$
    y_i = b_i - \left( L_{i,1} y_1 + L_{i,2} y_2 + \cdots + L_{i,i-1} y_{i-1} \right ).
$$

### Backward Substitution

Backwards substitution requires us to move from the last row of $U$ and move upwards.  We can consider again the general $i$th row with
$$
    U_{i,i} x_i + U_{i,i+1} x_{i+1} + \ldots + U_{i,m-1} x_{m-1} + U_{i,m} x_m = y_i.
$$
We can now solve for $x_i$ as
$$
    x_i = \frac{1}{U_{i,i}} \left( y_i - ( U_{i,i+1} x_{i+1} + \ldots + U_{i,m-1} x_{m-1} + U_{i,m} x_m) \right )
$$

### LU decomposition complexity

 * Backward/forward substitution has $\sum_{k=0}^{m} k = O(m^2)$ operations.
 * $LU$ factorization is in  the order of $O(m^3)$


